In [203]:
import os
import json
import pandas as pd
import traceback

In [204]:
from langchain.chat_models import ChatOpenAI

In [205]:
from dotenv import load_dotenv

load_dotenv()

True

In [206]:
KEY=os.getenv("OPENAI_API")

In [207]:
KEY

'sk-proj-AvRlOUoiJUCrTbwFezUoT3BlbkFJoKB3d2stsvL9HS9U2Tof'

In [208]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [209]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001CD3F06D7E0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001CD3F06F5B0>, temperature=0.5, openai_api_key='sk-proj-AvRlOUoiJUCrTbwFezUoT3BlbkFJoKB3d2stsvL9HS9U2Tof', openai_proxy='')

In [210]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [211]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [212]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [213]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [214]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [215]:
TEMPLATE2="""
You are an expert english grammarian and writer and an academic expert. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
If the quiz is not at par with the cognitive and analytical abilities of the students,\
Update the quiz questions which need to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from the point of view of an expert English Writer of the above quiz:
"""

In [216]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [217]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [218]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],output_variables=["quiz", "review"], verbose=True)

In [221]:
file_path=r"C:\Users\Mayank Chandak\MCQgenerator\data.txt"

In [222]:
file_path

'C:\\Users\\Mayank Chandak\\MCQgenerator\\data.txt'

In [223]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [225]:
print(TEXT)

Artificial Intelligence is the intelligence possessed by the machines under which they can perform various functions with human help. With the help of A.I, machines will be able to learn, solve problems, plan things, think, etc. Artificial Intelligence, for example, is the simulation of human intelligence by machines. In the field of technology, Artificial Intelligence is evolving rapidly day by day and it is believed that in the near future, artificial intelligence is going to change human life very drastically and will most probably end all the crises of the world by sorting out the major problems. 


Our life in this modern age depends largely on computers. It is almost impossible to think about life without computers. We need computers in everything that we use in our daily lives. So it becomes very important to make computers intelligent so that our lives become easy. Artificial Intelligence is the theory and development of computers, which imitates the human intelligence and sens

In [226]:
NUMBER=5 
SUBJECT="artificial intelligence"
TONE="simple"

In [229]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Artificial Intelligence is the intelligence possessed by the machines under which they can perform various functions with human help. With the help of A.I, machines will be able to learn, solve problems, plan things, think, etc. Artificial Intelligence, for example, is the simulation of human intelligence by machines. In the field of technology, Artificial Intelligence is evolving rapidly day by day and it is believed that in the near future, artificial intelligence is going to change human life very drastically and will most probably end all the crises of the world by sorting out the major problems. 


Our life in this modern age depends largely on computers. It is almost impossible to think about life without computers. We need computers in everything that we use in our daily lives. So it becomes very important to make computers intelligent so that our lives become easy. Artifi

In [231]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2532
Prompt Tokens:2052
Completion Tokens:480
Total Cost:0.004038000000000001


In [236]:
print(response.get("quiz"))


{
  "1": {
    "mcq": "What is the main goal of General AI (AGI) systems?",
    "options": {
      "a": "Perform a specific task efficiently",
      "b": "Think like a human on its own",
      "c": "React as per the best possible action",
      "d": "Store data for future actions"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which type of AI system can surpass human intelligence and perform any cognitive task better than humans?",
    "options": {
      "a": "Reactive Machines",
      "b": "Limited Memory",
      "c": "General AI (AGI)",
      "d": "Narrow AI"
    },
    "correct": "c"
  },
  "3": {
    "mcq": "What is the characteristic of Narrow AI (weak AI) systems?",
    "options": {
      "a": "Can think like a human on its own",
      "b": "Perform a specific task with intelligence",
      "c": "React as per the best possible action",
      "d": "Store data for future actions"
    },
    "correct": "b"
  },
  "4": {
    "mcq": "Which type of AI system focuses only on curr

In [270]:
quiz=response.get("quiz")
quiz=json.loads(quiz)

In [271]:
quiz

{'1': {'mcq': 'What is the main goal of General AI (AGI) systems?',
  'options': {'a': 'Perform a specific task efficiently',
   'b': 'Think like a human on its own',
   'c': 'React as per the best possible action',
   'd': 'Store data for future actions'},
  'correct': 'b'},
 '2': {'mcq': 'Which type of AI system can surpass human intelligence and perform any cognitive task better than humans?',
  'options': {'a': 'Reactive Machines',
   'b': 'Limited Memory',
   'c': 'General AI (AGI)',
   'd': 'Narrow AI'},
  'correct': 'c'},
 '3': {'mcq': 'What is the characteristic of Narrow AI (weak AI) systems?',
  'options': {'a': 'Can think like a human on its own',
   'b': 'Perform a specific task with intelligence',
   'c': 'React as per the best possible action',
   'd': 'Store data for future actions'},
  'correct': 'b'},
 '4': {'mcq': 'Which type of AI system focuses only on current situations and does not store memories for future actions?',
  'options': {'a': 'Limited Memory',
   'b': '

In [272]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join([f"{option}: {option_value}" for option, option_value in value["options"].items()])
    correct = value["correct"]
    quiz_table_data.append({"Question no.":key,"MCQ": mcq, "Choices": options, "Correct": correct})

In [273]:
quiz_table_data

[{'Question no.': '1',
  'MCQ': 'What is the main goal of General AI (AGI) systems?',
  'Choices': 'a: Perform a specific task efficiently | b: Think like a human on its own | c: React as per the best possible action | d: Store data for future actions',
  'Correct': 'b'},
 {'Question no.': '2',
  'MCQ': 'Which type of AI system can surpass human intelligence and perform any cognitive task better than humans?',
  'Choices': 'a: Reactive Machines | b: Limited Memory | c: General AI (AGI) | d: Narrow AI',
  'Correct': 'c'},
 {'Question no.': '3',
  'MCQ': 'What is the characteristic of Narrow AI (weak AI) systems?',
  'Choices': 'a: Can think like a human on its own | b: Perform a specific task with intelligence | c: React as per the best possible action | d: Store data for future actions',
  'Correct': 'b'},
 {'Question no.': '4',
  'MCQ': 'Which type of AI system focuses only on current situations and does not store memories for future actions?',
  'Choices': 'a: Limited Memory | b: Rea

In [274]:
quiz=pd.DataFrame(quiz_table_data)

In [275]:
quiz

,Question no.,MCQ,Choices,Correct
0,1,What is the main goal of General AI (AGI) syst...,a: Perform a specific task efficiently | b: Th...,b
1,2,Which type of AI system can surpass human inte...,a: Reactive Machines | b: Limited Memory | c: ...,c
2,3,What is the characteristic of Narrow AI (weak ...,a: Can think like a human on its own | b: Perf...,b
3,4,Which type of AI system focuses only on curren...,a: Limited Memory | b: Reactive Machines | c: ...,b
4,5,What is the main feature of Super AI systems?,a: Ability to think like a human on its own | ...,c


In [277]:
quiz.to_csv("AI_quiz.csv",index=False)